In [12]:
import os

In [13]:
%pwd

'm:\\study\\Xray project'

In [16]:
os.chdir('m:\\study\\Xray project')

In [39]:
%pwd

'm:\\study\\Xray project'

In [40]:
from dataclasses import dataclass
from pathlib import Path

__Model parameter types__

In [41]:
# for the model : Parameter TYPE class > Parameter Variable class

@dataclass(frozen=True)           # by setting frozen=True, we are restricting any changes in the class further
class PrepareBaseModelConfig:

    '''This class holds the required parameters types for model. @dataclass(frozen=True) set to True, so that the datatypes cann't be changed in further. If user will try to add anything new, it will throw an error'''

    #path parameters
    root_dir : Path
    base_model_path : Path
    base_updated_model_path : Path


    #model parameters
    params_image_size : list
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes : int

__Model Parameters value assign__

In [42]:
from cnnClassifier.constant import *
from cnnClassifier.utilis.common import read_yaml, create_directories


# for the model : Parameter VARIABLE class
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
    ):                                                  # initialization ends here
        
        #creating variable and reading the parameter values from the file at the mentioned path
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        #creating the directories
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self)-> PrepareBaseModelConfig :

        '''This function reads the data from the initialized parameters.
            
            Job : Assigning values to the PrepareBaseModelConfig mentioned variables
            returns : a class obj of PrepareBaseModelConfig'''

        #reading the values of the variables of the model
        config = self.config.prepare_base_model     
        # prepare_base_model : is a part in the yaml file, that contains values of the model variable

        #creating folder
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(

            #reading the values from the config.yaml file <prepare_base_model> part
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            base_updated_model_path = Path(config.updated_base_model_path),

            #reading the values from the params.yaml file 
            params_image_size = self.params.IMAGE_SIZE,
            params_learning_rate = self.params.LEARNING_RATE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS,
            params_classes = self.params.CLASSES)
        
        return prepare_base_model_config

__Creating the base model__

In [43]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [50]:
class PrepareBaseModel:

    #initializing
    def __init__(self, config : PrepareBaseModelConfig):
        self.config = config
    

    #downloading the VGG16 model
    def get_base_model(self):

        '''This function downloads the VGG16 model from the internet'''

        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights= self.config.params_weights,
            include_top= self.config.params_include_top,       
        )

    #saving the base model
          

        self.save_model(path = self.config.base_model_path, model = self.model)

    
    #updating the base model
    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):

        '''This model update the base model with mentioned modifications'''

        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    @staticmethod
    def save_model(path : Path, model : tf.keras.Model):
        '''This function saves the model'''
        model.save(path)
    
    #applying <prepare_full_model> function to create an updated model
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
    
        #saving the updated model
        self.save_model(path=self.config.base_updated_model_path, model = self.full_model)


__Pipelline__

In [51]:
try:

    '''Getting the configuration'''

    #creating an object of ConfigurationManager
    x = ConfigurationManager()
    
    #getting the configuration details
    prepare_base_model_config = x.get_prepare_base_model_config()


    '''Creating the model'''

    #initializing an object of PrepareBaseModel
    prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)

    #getting the base model
    prepare_base_model.get_base_model()

    #updating the base model
    prepare_base_model.update_base_model()


except Exception as e:
    raise e

[2024-04-29 21:04:32,854 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2024-04-29 21:04:32,858 : INFO : common : yaml file: params.yaml loaded successfully]
[2024-04-29 21:04:32,861 : INFO : common : created directory at: artifacts]
[2024-04-29 21:04:32,862 : INFO : common : created directory at: artifacts/prepare_base_model]
[2024-04-29 21:04:33,257 : WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (N